# EDA jobsearch about Säkerhet och bevakning

In [ ]:
import requests 
import json 


url = "https://jobsearch.api.jobtechdev.se"
url_for_search = f"{url}/search"


def _get_ads(params):
    headers = {'accept': 'application/json'}
    response = requests.get(url_for_search, headers=headers, params=params)
    response.raise_for_status()  # check for http errors
    return json.loads(response.content.decode('utf8'))


def example_search_return_number_of_hits(query):
    # limit: 0 means no ads, just a value of how many ads were found.
    search_params = {'q': query, 'limit': 0}
    json_response = _get_ads(search_params)
    number_of_hits = json_response['total']['value']
    print(f"\nNumber of hits = {number_of_hits}")


def example_search_loop_through_hits(query):
    # limit = 100 is the max number of hits that can be returned.
    # If there are more (which you find with ['total']['value'] in the json response)
    # you have to use offset and multiple requests to get all ads.
    search_params = {'q': query, 'limit': 100}
    json_response = _get_ads(search_params)
    hits = json_response['hits']
    for hit in hits:
        print(f"{hit['headline']}, {hit['employer']['name']}")


if __name__ == '__main__':
    query = 'säkerhet och bevakning'
    example_search_loop_through_hits(query)
    example_search_return_number_of_hits(query)
    
    

In [ ]:
search_params = {'q': query, 'limit': 100}
json_response = _get_ads(search_params)

json_response.keys()

In [ ]:
type(json_response)

In [ ]:
json_response["hits"]

In [ ]:
json_response["hits"][5]["headline"], json_response["hits"][5]["number_of_vacancies"], json_response["hits"][5]["salary_type"]["label"]

In [46]:
json_response["hits"][0]["employer"]

{'phone_number': None,
 'email': None,
 'url': None,
 'organization_number': '2120000365',
 'name': 'Strängnäs kommun',
 'workplace': 'Strängnäs kommun, Säkerhetsenheten'}

In [45]:
json_response["hits"][0].keys()

dict_keys(['relevance', 'id', 'external_id', 'original_id', 'label', 'webpage_url', 'logo_url', 'headline', 'application_deadline', 'number_of_vacancies', 'description', 'employment_type', 'salary_type', 'salary_description', 'duration', 'working_hours_type', 'scope_of_work', 'access', 'employer', 'application_details', 'experience_required', 'access_to_own_car', 'driving_license_required', 'driving_license', 'occupation', 'occupation_group', 'occupation_field', 'workplace_address', 'must_have', 'nice_to_have', 'application_contacts', 'publication_date', 'last_publication_date', 'removed', 'removed_date', 'source_type', 'timestamp'])

In [44]:
json_response["hits"][-1]["employer"]

{'phone_number': None,
 'email': None,
 'url': 'http://gvstars.se',
 'organization_number': '5593284838',
 'name': 'GV Stars Bemanning AB',
 'workplace': 'GV Stars Bemanning AB'}

In [84]:
json_response["hits"]

name = []
for i in json_response["hits"]:
    name.append(i["employment_type"]["label"])


print(name)

    

['Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Behovsanställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Behovsanställning', 'Vanlig anställning', 'Behovsanställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anställning', 'Vanlig anst

In [57]:
from pandas import json_normalize
import pandas as pd
data = json_response["hits"]
df = pd.json_normalize(data)
df.head()

,relevance,id,external_id,original_id,label,webpage_url,logo_url,headline,application_deadline,number_of_vacancies,salary_description,access,experience_required,access_to_own_car,driving_license_required,driving_license,application_contacts,publication_date,last_publication_date,removed,removed_date,source_type,timestamp,description.text,description.text_formatted,description.company_information,description.needs,description.requirements,description.conditions,employment_type.concept_id,employment_type.label,employment_type.legacy_ams_taxonomy_id,salary_type.concept_id,salary_type.label,salary_type.legacy_ams_taxonomy_id,duration.concept_id,duration.label,duration.legacy_ams_taxonomy_id,working_hours_type.concept_id,working_hours_type.label,...,employer.name,employer.workplace,application_details.information,application_details.reference,application_details.email,application_details.via_af,application_details.url,application_details.other,occupation.concept_id,occupation.label,occupation.legacy_ams_taxonomy_id,occupation_group.concept_id,occupation_group.label,occupation_group.legacy_ams_taxonomy_id,occupation_field.concept_id,occupation_field.label,occupation_field.legacy_ams_taxonomy_id,workplace_address.municipality,workplace_address.municipality_code,workplace_address.municipality_concept_id,workplace_address.region,workplace_address.region_code,workplace_address.region_concept_id,workplace_address.country,workplace_address.country_code,workplace_address.country_concept_id,workplace_address.street_address,workplace_address.postcode,workplace_address.city,workplace_address.coordinates,must_have.skills,must_have.languages,must_have.work_experiences,must_have.education,must_have.education_level,nice_to_have.skills,nice_to_have.languages,nice_to_have.work_experiences,nice_to_have.education,nice_to_have.education_level
0,1.000000,29858134,46-212000-0365-278A5F21FF7E4C10AE4EBFBDEB9359B7,None,[],https://arbetsformedlingen.se/platsbanken/anno...,https://arbetsformedlingen.se/rest/agas/api/v1...,Säkerhetssamordnare med inriktning teknisk bev...,2025-09-05T23:59:59,1,Enligt avtal.,None,False,False,False,None,"[{'name': None, 'description': 'Per Maniette',...",2025-08-01T00:00:00,2025-09-05T23:59:59,False,None,VIA_PLATSBANKEN_DXA,1755084629405,Vill du vara med och utveckla vårt trygga och ...,Vill du vara med och utveckla vårt trygga och ...,None,None,None,"Heltid. \r\nTillsvidareanställning, tillträde:...",PFZr_Syz_cUq,Vanlig anställning,1,oG8G_9cW_nRf,Fast månads- vecko- eller timlön,1,a7uU_j21_mkL,Tills vidare,1,6YE1_gAC_R2G,Heltid,...,Strängnäs kommun,"Strängnäs kommun, Säkerhetsenheten",None,219/25,None,False,https://recruit.visma.com/spa/public/apply?gui...,None,PLBW_6Py_neC,Säkerhetsrådgivare,6771,nuXg_Wjc_JPd,Säkerhetsinspektörer m.fl.,3354,E7hm_BLq_fqZ,Säkerhet och bevakning,17,Strängnäs,0486,shnD_RiE_RKL,Södermanlands län,04,s93u_BEb_sx2,Sverige,199,i46j_HmG_v64,None,None,None,"[17.03212, 59.377453]",[],[],[],[],[],[],[],[],[],[]
1,0.999986,29969775,46-556768-8006-1086,None,[],https://arbetsformedlingen.se/platsbanken/anno...,https://arbetsformedlingen.se/rest/agas/api/v1...,Parkeringsvakt till Göteborg,2025-10-31T23:59:59,1,Enligt kollektivavtal,None,True,False,False,None,[],2025-09-01T08:45:06,2025-10-31T23:59:59,False,None,VIA_PLATSBANKEN_DXA,1756709106375,QSG Bevakning erbjuder ett flertal tjänster ti...,QSG Bevakning erbjuder ett flertal tjänster ti...,None,None,None,Heltid\r\nTillsvidareanställning med 6 månader...,PFZr_Syz_cUq,Vanlig anställning,1,oG8G_9cW_nRf,Fast månads- vecko- eller timlön,1,a7uU_j21_mkL,Tills vidare,1,6YE1_gAC_R2G,Heltid,...,QSG Bevakning AB,QSG Bevakning,None,None,None,False,https://qsg.se/lediga-tjanster/1086,None,44uM_XPP_VdK,Parkeringsvakt,5907,ayQ4_G3W_A2w,Övrig bevaknings- och säkerhetspersonal,5419,E7hm_BLq_fqZ,Säkerhet och bevakning,17,Göteborg,1480,PVZL_BQT_XtL,Västra Götalands län,14,zdoY_6u5_Krt,Sverige,199,i46j_HmG_v64,None,None,None,"[11.97456, 57.70887]",[],[],"[{'weight': 10, 'conc

In [70]:
new_df = df[["id", "employer.name", "employer.workplace", "workplace_address.country", "number_of_vacancies", "timestamp", "occupation.label", "workplace_address.region"]]

new_df.head()

,id,employer.name,employer.workplace,workplace_address.country,number_of_vacancies,timestamp,occupation.label,workplace_address.region
0,29858134,Strängnäs kommun,"Strängnäs kommun, Säkerhetsenheten",Sverige,1,1755084629405,Säkerhetsrådgivare,Södermanlands län
1,29969775,QSG Bevakning AB,QSG Bevakning,Sverige,1,1756709106375,Parkeringsvakt,Västra Götalands län
2,29927239,Addici Security AB,Addici,Sverige,2,1755673285774,Väktare,Västra Götalands län
3,29927259,Addici Security AB,Addici,Sverige,2,1755673226213,Väktare,Västra Götalands län
4,29887183,Addici Security AB,Addici,Sverige,1,1755092527645,Stationär väktare/Receptionsväktare,Stockholms län


In [72]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         100 non-null    object
 1   employer.name              100 non-null    object
 2   employer.workplace         100 non-null    object
 3   workplace_address.country  100 non-null    object
 4   number_of_vacancies        100 non-null    int64 
 5   timestamp                  100 non-null    int64 
 6   occupation.label           100 non-null    object
 7   workplace_address.region   96 non-null     object
dtypes: int64(2), object(6)
memory usage: 6.4+ KB


In [86]:

# using groupby and sort to get out number available vacacancies
top_10_vacancies = new_df.groupby("employer.name")["number_of_vacancies"].sum().sort_values(ascending= False).reset_index(name= "number_vacancies")

top_10_vacancies = top_10_vacancies.head(10)
top_10_vacancies

,employer.name,number_vacancies
0,Prisma Security AB,44
1,Kriminalvården,29
2,Försvarsmakten,23
3,GV Stars Bemanning AB,12
4,Cubsec AB,7
5,Securitas AB,6
6,Commuter Security Group AB,6
7,Addici Security AB,5
8,QSG Bevakning AB,5
9,Skatteverket,4
